## Identifying and resolving conflicts in a batch of VI files

ELGs - part 1 is sets 2 to 4

448 visual inspections of a total of 150 unique objects

32 conflicts

In [1]:
save_filename='truth_table_ELGs_sets2_4_new.txt'

In [2]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

sys.path.append("/global/homes/r/rtojeiro/SV/python")
import VI_utils as VI

#import desispec
# library location will change ..
sys.path.append("/global/homes/r/rtojeiro/prospect/prospect/py")
from prospect import utils_specviewer,plotframes
import matplotlib.pyplot as plt 

In [3]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/'
tiles = ['67230']
nights = ['20200315']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']
obs_db = utils_specviewer.make_targetdict(tiledir, petals=petals, tiles=tiles) # tiles = optional 
del obs_db['67230-20200314']
obs_db.keys()

dict_keys(['67230-20200315'])

In [4]:
pd.set_option('display.max_rows', 10)

In [5]:
#set to directory with all the VI files to merge
VI_dir = os.environ['HOME']+'/SV/VI_files/SV0/ELG/'

In [6]:
vi_files = ['desi-vi_SV0_ELG_tile67230_night20200315_2_PNT.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_2_CBA.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_2_ace.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_3_CBA.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_3_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_3_PNT.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_4_ASC.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_4_VRK.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_4_ACE.csv']

In [7]:
vi = pd.read_csv(VI_dir + vi_files[0], delimiter = " , ", engine='python')

for i in range(1,len(vi_files)):
    print(vi_files[i])
    vi2 = pd.read_csv(VI_dir + vi_files[i], delimiter = " , ", engine='python')
    vi = vi.append(vi2, ignore_index=True)
    
#make groups of visual inspections, grouped by unique objects, and state number of single and multiple VIs
vi_gp = vi.groupby(['TargetID'])
print('There are ' + str(len(vi)) + ' visual inspections of a total of ' + str(len(vi_gp)) + ' unique objects')
print('That gives ' + str(len(vi)/len(vi_gp)) + ' inspections per object.')

desi-vi_SV0_ELG_tile67230_night20200315_2_CBA.csv
desi-vi_SV0_ELG_tile67230_night20200315_2_ace.csv
desi-vi_SV0_ELG_tile67230_night20200315_3_CBA.csv
desi-vi_SV0_ELG_tile67230_night20200315_3_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_3_PNT.csv
desi-vi_SV0_ELG_tile67230_night20200315_4_ASC.csv
desi-vi_SV0_ELG_tile67230_night20200315_4_VRK.csv
desi-vi_SV0_ELG_tile67230_night20200315_4_ACE.csv
There are 448 visual inspections of a total of 150 unique objects
That gives 2.986666666666667 inspections per object.


In [8]:
#vi is a dataframe
display(vi)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment
0,35185735929695223,-1,0.33.0.dev3890,0,GALAXY,1.0936,PNT,1,RCS,--,GALAXY,Difficult to asses. I think it is noisy spectra.
1,35185735929695459,-1,0.33.0.dev3890,0,GALAXY,0.8155,PNT,4,--,0.8155,GALAXY,--
2,35185735929695590,-1,0.33.0.dev3890,0,GALAXY,1.5142,PNT,1,RCS,0,GALAXY,Noisy spectra
3,35185735929695765,-1,0.33.0.dev3890,0,GALAXY,1.2587,PNT,4,--,1.2587,GALAXY,--
4,35185735929695797,-1,0.33.0.dev3890,0,GALAXY,1.1690,PNT,4,--,1.1690,GALAXY,--
...,...,...,...,...,...,...,...,...,...,...,...,...
443,35185741965300315,-1,0.33.0.dev3890,0,GALAXY,0.8725,ACE,4,--,--,--,--
444,35185741969490082,-1,0.33.0.dev3890,0,GALAXY,1.3515,ACE,0,--,--,--,OII on sky line not plausible
445,35185741969490342,-1,0.33.0.dev3890,0,QSO,1.9424,ACE,2,--,--,--,Broad line in the blue low signal to noise
446,35185741969490438,-1,0.33.0.dev3890,0,GALAXY,1.6280,ACE,0,--,--,--,No signal


In [9]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment'],
      dtype='object')

### Merge with zbest files
Add: fiberID, delta_chi2, flux information,.. anything else?


In [10]:

#read in fibermap info, loop over the files for all the petals
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

tspec_df = tspec['TARGETID','DELTACHI2' ].to_pandas()
tf_df = tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_pandas()

tf_df = tf_df.rename(columns={"TARGETID": "TargetID"})
tspec_df = tspec_df.rename(columns={"TARGETID": "TargetID"})

vi = vi.merge(tf_df, how='left', on='TargetID')
vi = vi.merge(tspec_df, how='left', on='TargetID')

In [11]:
print(vi.keys())
print(len(vi))

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2'],
      dtype='object')
448


### Adding a bunch of useful columns

In [12]:
vi = VI.prep_new_columns(vi)

In [13]:
#check all the new columns (keys) have been added correctly
display(vi)
print(vi.keys())

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
0,35185735929695223,-1,0.33.0.dev3890,0,GALAXY,1.0936,PNT,1,RCS,--,...,46.116570,1.0936,GALAXY,2.333333,3,0.0000,1.0936,Difficult to asses. I think it is noisy spect...,3,none
1,35185735929695459,-1,0.33.0.dev3890,0,GALAXY,0.8155,PNT,4,--,0.8155,...,959.128342,0.8155,GALAXY,4.000000,0,0.0000,0.8155,--| [OII] Hb [OIII]|--,3,none
2,35185735929695590,-1,0.33.0.dev3890,0,GALAXY,1.5142,PNT,1,RCS,0,...,1.790732,0.0000,GALAXY,0.333333,1,1.5142,1.5142,Noisy spectra| Bad sky subtraction not [OII]|...,3,none
3,35185735929695765,-1,0.33.0.dev3890,0,GALAXY,1.2587,PNT,4,--,1.2587,...,627.763939,1.2587,GALAXY,4.000000,0,0.0000,1.2587,--| [OII] doublet|--,3,none
4,35185735929695797,-1,0.33.0.dev3890,0,GALAXY,1.1690,PNT,4,--,1.1690,...,554.882680,1.1690,GALAXY,4.000000,0,0.0000,1.1690,--| [OII] doublet|--,3,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,35185741965300315,-1,0.33.0.dev3890,0,GALAXY,0.8725,ACE,4,--,--,...,227.709846,0.8725,GALAXY,4.000000,0,0.0000,0.8725,--|--|--,3,none
444,35185741969490082,-1,0.33.0.dev3890,0,GALAXY,1.3515,ACE,0,--,--,...,26.390474,1.3515,GALAXY,1.000000,3,0.0000,1.3515,--|--| OII on sky line not plausible,3,none
445,35185741969490342,-1,0.33.0.dev3890,0,QSO,1.9424,ACE,2,--,--,...,4.072479,1.9424,QSO,2.000000,2,0.0000,1.9424,Possible C IV|not an ELG spectrum + could be ...,3,none
446,35185741969490438,-1,0.33.0.dev3890,0,GALAXY,1.6280,ACE,0,--,--,...,0.852903,1.6280,GALAXY,0.333333,1,0.0000,1.6280,--|dubious doublet at the very end of the NIR ...,3,none


Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff', 'dz',
       'vi_combined_z', 'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')


Get a table that holds only the objects that have been inspected more than once, and for which the individual VI classifications differ by 2 or more, or delta z / (1 + z) > 0.0033, or there is disagreement in best spectype (these are the conflicts to resolve)

In [14]:
vi_gp = vi.groupby(['TargetID'])
vi_conflict = VI.find_conflicts(vi_gp)

Get the target IDs of the problematic objects and display in table form for a quick summary:

In [15]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185735929695223 35185735929696706 35185735929696789 35185741948523207
 35185741952713123 35185741952713852 35185741952714525 35185741952714607
 35185741952715552 35185741952716516 35185741952717119 35185741952717281
 35185741952717584 35185741952718121 35185741956908281 35185741956908529
 35185741956910226 35185741956911894 35185741956913486 35185741961101345
 35185741961101395 35185741961101910 35185741961101954 35185741961103043
 35185741961104153 35185741961104546 35185741961105562 35185741965295921
 35185741965297527 35185741965298041 35185741969490082 35185741969490342]
Total number of conflicts to resolve:  32


In [16]:
unique_targets[1]

35185735929696706

## This is where I resolve things manually - with care!!
### I think it's better to keep it in a notebook, as typos can be backtracked rather than a single manual edit of a text file

We edit either 'VI class', or 'best redshift' to resolve conflict. At the end, we look for conflicts again and we should find none.


In [17]:
#function to display the conflict in table format and open a prospect window
def display_conflict(conflict_id, show_spectra=None):
    
    #first, remind myself of the problem:
    display(vi[vi.TargetID==unique_targets[conflict_id]][['TargetID', 
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff',
       'vi_combined_z', 'dz', 'all VI comments', 'N_VI', 'merger comment']])

    spectra, zcat= utils_specviewer.load_spectra_zcat_from_targets([unique_targets[conflict_id]], tiledir, obs_db)
    # VI interface in notebook
    if show_spectra:
        plotframes.plotspectra(spectra, zcatalog=zcat, title='Target_select', notebook=True, mask_type='CMX_TARGET',with_vi_widgets=False)

In [18]:
#first, keep a safe copy of the original dataframe
vi_safe = vi.copy()

We will inspect each conflict on a prospect window, and resolve each conflict in turn

In [19]:
#Keep track of the conflicts by conflict_id

#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
0,35185735929695223,GALAXY,1.0936,PNT,1,RCS,--,GALAXY,Difficult to asses. I think it is noisy spectra.,1.0936,GALAXY,2.333333,3,1.0936,0.0,Difficult to asses. I think it is noisy spect...,3,none
48,35185735929695223,GALAXY,1.0936,CBA,2,--,--,--,Possible [OII],1.0936,GALAXY,2.333333,3,1.0936,0.0,Difficult to asses. I think it is noisy spect...,3,none
98,35185735929695223,GALAXY,1.0936,ace,4,--,--,--,--,1.0936,GALAXY,2.333333,3,1.0936,0.0,Difficult to asses. I think it is noisy spect...,3,none


In [20]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. broadened but unresolved line. not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
0,35185735929695223,-1,0.33.0.dev3890,0,GALAXY,1.0936,PNT,2.333333,RCS,--,...,46.11657,1.0936,GALAXY,2.333333,3,0.0,1.0936,Difficult to asses. I think it is noisy spect...,3,disagreement in flag. broadened but unresolved...
48,35185735929695223,-1,0.33.0.dev3890,0,GALAXY,1.0936,CBA,2.333333,--,--,...,46.11657,1.0936,GALAXY,2.333333,3,0.0,1.0936,Difficult to asses. I think it is noisy spect...,3,disagreement in flag. broadened but unresolved...
98,35185735929695223,-1,0.33.0.dev3890,0,GALAXY,1.0936,ace,2.333333,--,--,...,46.11657,1.0936,GALAXY,2.333333,3,0.0,1.0936,Difficult to asses. I think it is noisy spect...,3,disagreement in flag. broadened but unresolved...


conflict ID =  0


In [21]:
#copy this text to a new cell to display the conflict
conflict_id=1
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
9,35185735929696706,GALAXY,0.001,PNT,2.0,C,0.0010,STAR,Seems like a STAR but it was classified as ga...,0.001,STAR,3.333333,2,0.001,0.0,Seems like a STAR but it was classified as ga...,3,none
57,35185735929696706,GALAXY,0.001,CBA,4.0,C,--,STAR,Star spectrum,0.001,STAR,3.333333,2,0.001,0.0,Seems like a STAR but it was classified as ga...,3,none
107,35185735929696706,GALAXY,0.001,ace,4.0,--,--,STAR,White dwarf - no a galaxy!,0.001,STAR,3.333333,2,0.001,0.0,Seems like a STAR but it was classified as ga...,3,none


In [22]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
9,35185735929696706,-1,0.33.0.dev3890,0,GALAXY,0.001,PNT,3.333333,C,0.0010,...,1665.573065,0.001,STAR,3.333333,2,0.0,0.001,Seems like a STAR but it was classified as ga...,3,"flag disagreement, take mean."
57,35185735929696706,-1,0.33.0.dev3890,0,GALAXY,0.001,CBA,3.333333,C,--,...,1665.573065,0.001,STAR,3.333333,2,0.0,0.001,Seems like a STAR but it was classified as ga...,3,"flag disagreement, take mean."
107,35185735929696706,-1,0.33.0.dev3890,0,GALAXY,0.001,ace,3.333333,--,--,...,1665.573065,0.001,STAR,3.333333,2,0.0,0.001,Seems like a STAR but it was classified as ga...,3,"flag disagreement, take mean."


conflict ID =  1


In [23]:
#copy this text to a new cell to display the conflict
conflict_id=2
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
10,35185735929696789,GALAXY,1.6682,PNT,4.0,RCS,0,GALAXY,Noisy spectra,0.0000,GALAXY,1.333333,4,1.6682,1.6682,Noisy spectra|--| no signal,3,none
58,35185735929696789,GALAXY,1.6682,CBA,0.0,--,--,--,--,1.6682,GALAXY,1.333333,4,1.6682,1.6682,Noisy spectra|--| no signal,3,none
108,35185735929696789,GALAXY,1.6682,ace,0.0,--,--,--,no signal,1.6682,GALAXY,1.333333,4,1.6682,1.6682,Noisy spectra|--| no signal,3,none


In [24]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.6682
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag.z not secure, low SNR. flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
10,35185735929696789,-1,0.33.0.dev3890,0,GALAXY,1.6682,PNT,1.0,RCS,0,...,3.730618,1.6682,GALAXY,1.333333,4,1.6682,1.6682,Noisy spectra|--| no signal,3,"disagreement in flag.z not secure, low SNR. fl..."
58,35185735929696789,-1,0.33.0.dev3890,0,GALAXY,1.6682,CBA,1.0,--,--,...,3.730618,1.6682,GALAXY,1.333333,4,1.6682,1.6682,Noisy spectra|--| no signal,3,"disagreement in flag.z not secure, low SNR. fl..."
108,35185735929696789,-1,0.33.0.dev3890,0,GALAXY,1.6682,ace,1.0,--,--,...,3.730618,1.6682,GALAXY,1.333333,4,1.6682,1.6682,Noisy spectra|--| no signal,3,"disagreement in flag.z not secure, low SNR. fl..."


conflict ID =  2


In [25]:
#copy this text to a new cell to display the conflict
conflict_id=3
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
25,35185741948523207,GALAXY,0.5128,PNT,4.0,RCS,0,STAR,Noisy spectra probably STAR,0.0000,STAR,1.333333,4,0.5128,0.5128,Noisy spectra probably STAR|--|no signal,3,none
73,35185741948523207,GALAXY,0.5128,CBA,0.0,--,--,--,--,0.5128,GALAXY,1.333333,4,0.5128,0.5128,Noisy spectra probably STAR|--|no signal,3,none
123,35185741948523207,GALAXY,0.5128,ace,0.0,--,--,--,no signal,0.5128,GALAXY,1.333333,4,0.5128,0.5128,Noisy spectra probably STAR|--|no signal,3,none


In [26]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.5128
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY' 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in all. low SNR, take RR answer with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
25,35185741948523207,-1,0.33.0.dev3890,0,GALAXY,0.5128,PNT,1.0,RCS,0,...,3.061283,0.5128,GALAXY,1.333333,4,0.5128,0.5128,Noisy spectra probably STAR|--|no signal,3,"disagreement in all. low SNR, take RR answer w..."
73,35185741948523207,-1,0.33.0.dev3890,0,GALAXY,0.5128,CBA,1.0,--,--,...,3.061283,0.5128,GALAXY,1.333333,4,0.5128,0.5128,Noisy spectra probably STAR|--|no signal,3,"disagreement in all. low SNR, take RR answer w..."
123,35185741948523207,-1,0.33.0.dev3890,0,GALAXY,0.5128,ace,1.0,--,--,...,3.061283,0.5128,GALAXY,1.333333,4,0.5128,0.5128,Noisy spectra probably STAR|--|no signal,3,"disagreement in all. low SNR, take RR answer w..."


conflict ID =  3


In [27]:
#copy this text to a new cell to display the conflict
conflict_id=4
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
29,35185741952713123,GALAXY,0.2835,PNT,2.0,RCS,0.2835,STAR,Noisy spectra,0.2835,STAR,2.333333,1,0.2835,0.808337,Noisy spectra| I suggest this is bad z fit by...,3,none
78,35185741952713123,GALAXY,0.2835,CBA,2.0,R,1.3209,--,I suggest this is bad z fit by RR [OII] doub...,1.3209,GALAXY,2.333333,1,0.2835,0.808337,Noisy spectra| I suggest this is bad z fit by...,3,none
128,35185741952713123,GALAXY,0.2835,ace,3.0,--,1.3210,GALAXY,Redrock misses and obvious OII line,1.3210,GALAXY,2.333333,1,0.2835,0.808337,Noisy spectra| I suggest this is bad z fit by...,3,none


In [28]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.3210
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. 2/3 inspectors suggest VI z based on missed resolved OII.take with their mean flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
29,35185741952713123,-1,0.33.0.dev3890,0,GALAXY,0.2835,PNT,2.5,RCS,0.2835,...,4.530503,1.321,STAR,2.333333,1,0.808337,0.2835,Noisy spectra| I suggest this is bad z fit by...,3,disagreement in flag and z. 2/3 inspectors sug...
78,35185741952713123,-1,0.33.0.dev3890,0,GALAXY,0.2835,CBA,2.5,R,1.3209,...,4.530503,1.321,GALAXY,2.333333,1,0.808337,0.2835,Noisy spectra| I suggest this is bad z fit by...,3,disagreement in flag and z. 2/3 inspectors sug...
128,35185741952713123,-1,0.33.0.dev3890,0,GALAXY,0.2835,ace,2.5,--,1.3210,...,4.530503,1.321,GALAXY,2.333333,1,0.808337,0.2835,Noisy spectra| I suggest this is bad z fit by...,3,disagreement in flag and z. 2/3 inspectors sug...


conflict ID =  4


In [29]:
#copy this text to a new cell to display the conflict
conflict_id=5
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
32,35185741952713852,GALAXY,1.1492,PNT,2.0,S,1.1492,GALAXY,"OII double is not quite resolved"","" the rest ...",1.1492,GALAXY,3.0,2,1.1492,0.0,"OII double is not quite resolved"","" the rest ...",3,none
81,35185741952713852,GALAXY,1.1492,CBA,4.0,S,--,--,Flux drop at 5800A. [OII] + abs features.,1.1492,GALAXY,3.0,2,1.1492,0.0,"OII double is not quite resolved"","" the rest ...",3,none
131,35185741952713852,GALAXY,1.1492,ace,3.0,--,--,--,--,1.1492,GALAXY,3.0,2,1.1492,0.0,"OII double is not quite resolved"","" the rest ...",3,none


In [30]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag but z is secure. calibration issues.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
32,35185741952713852,-1,0.33.0.dev3890,0,GALAXY,1.1492,PNT,3.0,S,1.1492,...,530.271635,1.1492,GALAXY,3.0,2,0.0,1.1492,"OII double is not quite resolved"","" the rest ...",3,disagreement in flag but z is secure. calibrat...
81,35185741952713852,-1,0.33.0.dev3890,0,GALAXY,1.1492,CBA,3.0,S,--,...,530.271635,1.1492,GALAXY,3.0,2,0.0,1.1492,"OII double is not quite resolved"","" the rest ...",3,disagreement in flag but z is secure. calibrat...
131,35185741952713852,-1,0.33.0.dev3890,0,GALAXY,1.1492,ace,3.0,--,--,...,530.271635,1.1492,GALAXY,3.0,2,0.0,1.1492,"OII double is not quite resolved"","" the rest ...",3,disagreement in flag but z is secure. calibrat...


conflict ID =  5


In [31]:
#copy this text to a new cell to display the conflict
conflict_id=6
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
35,35185741952714525,GALAXY,1.5,PNT,2.0,RCS,2.0000,GALAXY,Noisy spectra only the OII doublet is resolve...,2.0000,GALAXY,1.666667,3,1.5,0.200336,Noisy spectra only the OII doublet is resolve...,3,none
85,35185741952714525,GALAXY,1.5,CBA,0.0,--,--,--,Unconvincing [OII] doublet,1.5000,GALAXY,1.666667,3,1.5,0.200336,Noisy spectra only the OII doublet is resolve...,3,none
135,35185741952714525,GALAXY,1.5,ace,3.0,--,1.4993,GALAXY,00 redshift bug at work here!,1.4993,GALAXY,1.666667,3,1.5,0.200336,Noisy spectra only the OII doublet is resolve...,3,none


In [33]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.5
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '00 bug.z not secure,flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
35,35185741952714525,-1,0.33.0.dev3890,0,GALAXY,1.5,PNT,1.0,RCS,2.0000,...,156.781308,1.5,GALAXY,1.666667,3,0.200336,1.5,Noisy spectra only the OII doublet is resolve...,3,"00 bug.z not secure,flag 1."
85,35185741952714525,-1,0.33.0.dev3890,0,GALAXY,1.5,CBA,1.0,--,--,...,156.781308,1.5,GALAXY,1.666667,3,0.200336,1.5,Noisy spectra only the OII doublet is resolve...,3,"00 bug.z not secure,flag 1."
135,35185741952714525,-1,0.33.0.dev3890,0,GALAXY,1.5,ace,1.0,--,1.4993,...,156.781308,1.5,GALAXY,1.666667,3,0.200336,1.5,Noisy spectra only the OII doublet is resolve...,3,"00 bug.z not secure,flag 1."


conflict ID =  6


In [36]:
#copy this text to a new cell to display the conflict
conflict_id=7
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
36,35185741952714607,GALAXY,1.164,PNT,4.0,RCS,0,STAR,Noisy data of a STAR probably,0.000,STAR,3.333333,1,1.164,1.164,Noisy data of a STAR probably| Probable [OII]|--,3,none
86,35185741952714607,GALAXY,1.164,CBA,3.0,--,--,--,Probable [OII],1.164,GALAXY,3.333333,1,1.164,1.164,Noisy data of a STAR probably| Probable [OII]|--,3,none
136,35185741952714607,GALAXY,1.164,ace,3.0,--,--,--,--,1.164,GALAXY,3.333333,1,1.164,1.164,Noisy data of a STAR probably| Probable [OII]|--,3,none


In [37]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.164
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and spectype. RR answer correct.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
36,35185741952714607,-1,0.33.0.dev3890,0,GALAXY,1.164,PNT,3.0,RCS,0,...,97.596352,1.164,GALAXY,3.333333,1,1.164,1.164,Noisy data of a STAR probably| Probable [OII]|--,3,disagreement in flag and spectype. RR answer c...
86,35185741952714607,-1,0.33.0.dev3890,0,GALAXY,1.164,CBA,3.0,--,--,...,97.596352,1.164,GALAXY,3.333333,1,1.164,1.164,Noisy data of a STAR probably| Probable [OII]|--,3,disagreement in flag and spectype. RR answer c...
136,35185741952714607,-1,0.33.0.dev3890,0,GALAXY,1.164,ace,3.0,--,--,...,97.596352,1.164,GALAXY,3.333333,1,1.164,1.164,Noisy data of a STAR probably| Probable [OII]|--,3,disagreement in flag and spectype. RR answer c...


conflict ID =  7


In [40]:
#copy this text to a new cell to display the conflict
conflict_id=8
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
42,35185741952715552,GALAXY,1.2701,PNT,4.0,--,1.2701,GALAXY,--,1.2701,GALAXY,3.0,2,1.2701,0.0,--| Possible [OII]|--,3,none
92,35185741952715552,GALAXY,1.2701,CBA,2.0,--,--,--,Possible [OII],1.2701,GALAXY,3.0,2,1.2701,0.0,--| Possible [OII]|--,3,none
142,35185741952715552,GALAXY,1.2701,ace,3.0,--,--,--,--,1.2701,GALAXY,3.0,2,1.2701,0.0,--| Possible [OII]|--,3,none


In [41]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'dubious OII. poor sky subtraction. calibration issues. z not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
42,35185741952715552,-1,0.33.0.dev3890,0,GALAXY,1.2701,PNT,2.0,--,1.2701,...,155.116283,1.2701,GALAXY,3.0,2,0.0,1.2701,--| Possible [OII]|--,3,dubious OII. poor sky subtraction. calibration...
92,35185741952715552,-1,0.33.0.dev3890,0,GALAXY,1.2701,CBA,2.0,--,--,...,155.116283,1.2701,GALAXY,3.0,2,0.0,1.2701,--| Possible [OII]|--,3,dubious OII. poor sky subtraction. calibration...
142,35185741952715552,-1,0.33.0.dev3890,0,GALAXY,1.2701,ace,2.0,--,--,...,155.116283,1.2701,GALAXY,3.0,2,0.0,1.2701,--| Possible [OII]|--,3,dubious OII. poor sky subtraction. calibration...


conflict ID =  8


In [46]:
#copy this text to a new cell to display the conflict
conflict_id=9
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
46,35185741952716516,GALAXY,1.0625,PNT,4.0,--,1.0625,GALAXY,--,1.0625,GALAXY,3.0,2,1.0625,0.0,--| Possible [OII]|--,3,none
96,35185741952716516,GALAXY,1.0625,CBA,2.0,--,--,--,Possible [OII],1.0625,GALAXY,3.0,2,1.0625,0.0,--| Possible [OII]|--,3,none
146,35185741952716516,GALAXY,1.0625,ace,3.0,--,--,--,--,1.0625,GALAXY,3.0,2,1.0625,0.0,--| Possible [OII]|--,3,none


In [47]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. OII and balmer lines, z is secure. poor sky subtraction. broadened but unresolved line.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
46,35185741952716516,-1,0.33.0.dev3890,0,GALAXY,1.0625,PNT,3.0,--,1.0625,...,59.408643,1.0625,GALAXY,3.0,2,0.0,1.0625,--| Possible [OII]|--,3,"disagreement in flag. OII and balmer lines, z ..."
96,35185741952716516,-1,0.33.0.dev3890,0,GALAXY,1.0625,CBA,3.0,--,--,...,59.408643,1.0625,GALAXY,3.0,2,0.0,1.0625,--| Possible [OII]|--,3,"disagreement in flag. OII and balmer lines, z ..."
146,35185741952716516,-1,0.33.0.dev3890,0,GALAXY,1.0625,ace,3.0,--,--,...,59.408643,1.0625,GALAXY,3.0,2,0.0,1.0625,--| Possible [OII]|--,3,"disagreement in flag. OII and balmer lines, z ..."


conflict ID =  9


In [48]:
#copy this text to a new cell to display the conflict
conflict_id=10
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
183,35185741952717119,GALAXY,1.3237,CBA,0.0,--,--,--,--,1.3237,GALAXY,1.333333,4,1.3237,0.0,--|No signal| Noisy data,3,none
233,35185741952717119,GALAXY,1.3237,ACE,0.0,--,--,--,No signal,1.3237,GALAXY,1.333333,4,1.3237,0.0,--|No signal| Noisy data,3,none
283,35185741952717119,GALAXY,1.3237,PNT,4.0,RCS,--,--,Noisy data,1.3237,GALAXY,1.333333,4,1.3237,0.0,--|No signal| Noisy data,3,none


In [49]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. low snr. flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
183,35185741952717119,-1,0.33.0.dev3890,0,GALAXY,1.3237,CBA,1.0,--,--,...,2.501833,1.3237,GALAXY,1.333333,4,0.0,1.3237,--|No signal| Noisy data,3,flag disagreement. low snr. flag 1.
233,35185741952717119,-1,0.33.0.dev3890,0,GALAXY,1.3237,ACE,1.0,--,--,...,2.501833,1.3237,GALAXY,1.333333,4,0.0,1.3237,--|No signal| Noisy data,3,flag disagreement. low snr. flag 1.
283,35185741952717119,-1,0.33.0.dev3890,0,GALAXY,1.3237,PNT,1.0,RCS,--,...,2.501833,1.3237,GALAXY,1.333333,4,0.0,1.3237,--|No signal| Noisy data,3,flag disagreement. low snr. flag 1.


conflict ID =  10


In [52]:
#copy this text to a new cell to display the conflict
conflict_id=11
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
185,35185741952717281,GALAXY,1.2253,CBA,4.0,--,--,--,[OII] doublet,1.2253,GALAXY,3.0,2,1.2253,0.0,[OII] doublet|--| Noisy data,3,none
235,35185741952717281,GALAXY,1.2253,ACE,3.0,--,--,--,--,1.2253,GALAXY,3.0,2,1.2253,0.0,[OII] doublet|--| Noisy data,3,none
285,35185741952717281,GALAXY,1.2253,PNT,2.0,--,1.2253,GALAXY,Noisy data,1.2253,GALAXY,3.0,2,1.2253,0.0,[OII] doublet|--| Noisy data,3,none


In [53]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. OII in skyline, is not secure. no continuum. poor sky subtraction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
185,35185741952717281,-1,0.33.0.dev3890,0,GALAXY,1.2253,CBA,2.0,--,--,...,41.604283,1.2253,GALAXY,3.0,2,0.0,1.2253,[OII] doublet|--| Noisy data,3,"disagreement in flag. OII in skyline, is not s..."
235,35185741952717281,-1,0.33.0.dev3890,0,GALAXY,1.2253,ACE,2.0,--,--,...,41.604283,1.2253,GALAXY,3.0,2,0.0,1.2253,[OII] doublet|--| Noisy data,3,"disagreement in flag. OII in skyline, is not s..."
285,35185741952717281,-1,0.33.0.dev3890,0,GALAXY,1.2253,PNT,2.0,--,1.2253,...,41.604283,1.2253,GALAXY,3.0,2,0.0,1.2253,[OII] doublet|--| Noisy data,3,"disagreement in flag. OII in skyline, is not s..."


conflict ID =  11


In [54]:
#copy this text to a new cell to display the conflict
conflict_id=12
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
186,35185741952717584,GALAXY,1.6176,CBA,0.0,--,--,--,--,1.6176,GALAXY,1.333333,4,1.6176,1.6176,--| No signal| Noisy spectra,3,none
236,35185741952717584,GALAXY,1.6176,ACE,0.0,--,--,--,No signal,1.6176,GALAXY,1.333333,4,1.6176,1.6176,--| No signal| Noisy spectra,3,none
286,35185741952717584,GALAXY,1.6176,PNT,4.0,--,0,--,Noisy spectra,0.0000,GALAXY,1.333333,4,1.6176,1.6176,--| No signal| Noisy spectra,3,none


In [55]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.6176
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. no signal. RR answer with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
186,35185741952717584,-1,0.33.0.dev3890,0,GALAXY,1.6176,CBA,1.0,--,--,...,12.46709,1.6176,GALAXY,1.333333,4,1.6176,1.6176,--| No signal| Noisy spectra,3,disagreement in flag and z. no signal. RR answ...
236,35185741952717584,-1,0.33.0.dev3890,0,GALAXY,1.6176,ACE,1.0,--,--,...,12.46709,1.6176,GALAXY,1.333333,4,1.6176,1.6176,--| No signal| Noisy spectra,3,disagreement in flag and z. no signal. RR answ...
286,35185741952717584,-1,0.33.0.dev3890,0,GALAXY,1.6176,PNT,1.0,--,0,...,12.46709,1.6176,GALAXY,1.333333,4,1.6176,1.6176,--| No signal| Noisy spectra,3,disagreement in flag and z. no signal. RR answ...


conflict ID =  12


In [58]:
#copy this text to a new cell to display the conflict
conflict_id=13
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
189,35185741952718121,GALAXY,0.5543,CBA,1.0,--,--,--,[OII] on bad sky subtraction,0.5543,GALAXY,1.666667,4,0.5543,0.0,[OII] on bad sky subtraction| No signal| Noisy...,3,none
239,35185741952718121,GALAXY,0.5543,ACE,0.0,--,--,--,No signal,0.5543,GALAXY,1.666667,4,0.5543,0.0,[OII] on bad sky subtraction| No signal| Noisy...,3,none
289,35185741952718121,GALAXY,0.5543,PNT,4.0,RCS,--,--,Noisy data,0.5543,GALAXY,1.666667,4,0.5543,0.0,[OII] on bad sky subtraction| No signal| Noisy...,3,none


In [59]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. very low snr. flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
189,35185741952718121,-1,0.33.0.dev3890,0,GALAXY,0.5543,CBA,1.0,--,--,...,13.147627,0.5543,GALAXY,1.666667,4,0.0,0.5543,[OII] on bad sky subtraction| No signal| Noisy...,3,disagreement in flag. very low snr. flag 1.
239,35185741952718121,-1,0.33.0.dev3890,0,GALAXY,0.5543,ACE,1.0,--,--,...,13.147627,0.5543,GALAXY,1.666667,4,0.0,0.5543,[OII] on bad sky subtraction| No signal| Noisy...,3,disagreement in flag. very low snr. flag 1.
289,35185741952718121,-1,0.33.0.dev3890,0,GALAXY,0.5543,PNT,1.0,RCS,--,...,13.147627,0.5543,GALAXY,1.666667,4,0.0,0.5543,[OII] on bad sky subtraction| No signal| Noisy...,3,disagreement in flag. very low snr. flag 1.


conflict ID =  13


In [62]:
#copy this text to a new cell to display the conflict
conflict_id=14
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
148,35185741956908281,GALAXY,1.5639,CBA,3.0,--,--,--,--,1.5639,GALAXY,3.333333,1,1.5639,1.5639,--|--|Noisy spectra,3,none
198,35185741956908281,GALAXY,1.5639,ACE,3.0,--,--,--,--,1.5639,GALAXY,3.333333,1,1.5639,1.5639,--|--|Noisy spectra,3,none
248,35185741956908281,GALAXY,1.5639,PNT,4.0,RC,0,--,Noisy spectra,0.0000,GALAXY,3.333333,1,1.5639,1.5639,--|--|Noisy spectra,3,none


In [63]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'one inspector set z to 0. broadened but unresolved line,z not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
148,35185741956908281,-1,0.33.0.dev3890,0,GALAXY,1.5639,CBA,2.0,--,--,...,71.439283,1.5639,GALAXY,3.333333,1,1.5639,1.5639,--|--|Noisy spectra,3,one inspector set z to 0. broadened but unreso...
198,35185741956908281,-1,0.33.0.dev3890,0,GALAXY,1.5639,ACE,2.0,--,--,...,71.439283,1.5639,GALAXY,3.333333,1,1.5639,1.5639,--|--|Noisy spectra,3,one inspector set z to 0. broadened but unreso...
248,35185741956908281,-1,0.33.0.dev3890,0,GALAXY,1.5639,PNT,2.0,RC,0,...,71.439283,0.0000,GALAXY,3.333333,1,1.5639,1.5639,--|--|Noisy spectra,3,one inspector set z to 0. broadened but unreso...


conflict ID =  14


In [66]:
#copy this text to a new cell to display the conflict
conflict_id=15
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
149,35185741956908529,GALAXY,0.9093,CBA,4.0,--,--,--,[OII] Balmer lines one [OIII],0.9093,GALAXY,3.333333,2,0.9093,0.9093,[OII] Balmer lines one [OIII]|--| Noisy data ...,3,none
199,35185741956908529,GALAXY,0.9093,ACE,4.0,--,--,--,--,0.9093,GALAXY,3.333333,2,0.9093,0.9093,[OII] Balmer lines one [OIII]|--| Noisy data ...,3,none
249,35185741956908529,GALAXY,0.9093,PNT,2.0,RCS,0,STAR,Noisy data Bad classification no noise is reg...,0.0000,STAR,3.333333,2,0.9093,0.9093,[OII] Balmer lines one [OIII]|--| Noisy data ...,3,none


In [67]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9093
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in spectype and z and flag. RR z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
149,35185741956908529,-1,0.33.0.dev3890,0,GALAXY,0.9093,CBA,4.0,--,--,...,299.710453,0.9093,GALAXY,3.333333,2,0.9093,0.9093,[OII] Balmer lines one [OIII]|--| Noisy data ...,3,disagreement in spectype and z and flag. RR z ...
199,35185741956908529,-1,0.33.0.dev3890,0,GALAXY,0.9093,ACE,4.0,--,--,...,299.710453,0.9093,GALAXY,3.333333,2,0.9093,0.9093,[OII] Balmer lines one [OIII]|--| Noisy data ...,3,disagreement in spectype and z and flag. RR z ...
249,35185741956908529,-1,0.33.0.dev3890,0,GALAXY,0.9093,PNT,4.0,RCS,0,...,299.710453,0.9093,GALAXY,3.333333,2,0.9093,0.9093,[OII] Balmer lines one [OIII]|--| Noisy data ...,3,disagreement in spectype and z and flag. RR z ...


conflict ID =  15


In [68]:
#copy this text to a new cell to display the conflict
conflict_id=16
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
192,35185741956910226,GALAXY,0.2409,CBA,2.0,--,--,--,Absortion features + continuum,0.2409,GALAXY,2.0,4,0.2409,0.2409,Absortion features + continuum| No features i...,3,none
242,35185741956910226,GALAXY,0.2409,ACE,0.0,--,--,--,No features in the weak continuum.,0.2409,GALAXY,2.0,4,0.2409,0.2409,Absortion features + continuum| No features i...,3,none
292,35185741956910226,GALAXY,0.2409,PNT,4.0,RCS,0,--,Noisy spectra,0.0000,GALAXY,2.0,4,0.2409,0.2409,Absortion features + continuum| No features i...,3,none


In [69]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.2409
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, z not secure. one inspector set z to 0'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
192,35185741956910226,-1,0.33.0.dev3890,0,GALAXY,0.2409,CBA,1.0,--,--,...,0.750252,0.2409,GALAXY,2.0,4,0.2409,0.2409,Absortion features + continuum| No features i...,3,"disagreement in flag, z not secure. one inspec..."
242,35185741956910226,-1,0.33.0.dev3890,0,GALAXY,0.2409,ACE,1.0,--,--,...,0.750252,0.2409,GALAXY,2.0,4,0.2409,0.2409,Absortion features + continuum| No features i...,3,"disagreement in flag, z not secure. one inspec..."
292,35185741956910226,-1,0.33.0.dev3890,0,GALAXY,0.2409,PNT,1.0,RCS,0,...,0.750252,0.2409,GALAXY,2.0,4,0.2409,0.2409,Absortion features + continuum| No features i...,3,"disagreement in flag, z not secure. one inspec..."


conflict ID =  16


In [70]:
#copy this text to a new cell to display the conflict
conflict_id=17
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
159,35185741956911894,GALAXY,0.26,CBA,4.0,--,--,--,[OII] Hb [OIII] Ha,0.26,GALAXY,4.0,0,0.26,0.015873,[OII] Hb [OIII] Ha|--| Noisy data major lines...,3,none
209,35185741956911894,GALAXY,0.26,ACE,4.0,--,--,--,--,0.26,GALAXY,4.0,0,0.26,0.015873,[OII] Hb [OIII] Ha|--| Noisy data major lines...,3,none
259,35185741956911894,GALAXY,0.26,PNT,4.0,--,0.2800,GALAXY,Noisy data major lines OII OIII Ha are resolved,0.28,GALAXY,4.0,0,0.26,0.015873,[OII] Hb [OIII] Ha|--| Noisy data major lines...,3,none


In [71]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.26
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '00 bug'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
159,35185741956911894,-1,0.33.0.dev3890,0,GALAXY,0.26,CBA,4.0,--,--,...,525.671865,0.26,GALAXY,4.0,0,0.015873,0.26,[OII] Hb [OIII] Ha|--| Noisy data major lines...,3,00 bug
209,35185741956911894,-1,0.33.0.dev3890,0,GALAXY,0.26,ACE,4.0,--,--,...,525.671865,0.26,GALAXY,4.0,0,0.015873,0.26,[OII] Hb [OIII] Ha|--| Noisy data major lines...,3,00 bug
259,35185741956911894,-1,0.33.0.dev3890,0,GALAXY,0.26,PNT,4.0,--,0.2800,...,525.671865,0.26,GALAXY,4.0,0,0.015873,0.26,[OII] Hb [OIII] Ha|--| Noisy data major lines...,3,00 bug


conflict ID =  17


In [72]:
#copy this text to a new cell to display the conflict
conflict_id=18
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
162,35185741956913486,GALAXY,0.7814,CBA,0.0,--,--,--,--,0.7814,GALAXY,1.0,3,0.7814,0.7814,--| No signal| Noisy data,3,none
212,35185741956913486,GALAXY,0.7814,ACE,0.0,--,--,--,No signal,0.7814,GALAXY,1.0,3,0.7814,0.7814,--| No signal| Noisy data,3,none
262,35185741956913486,GALAXY,0.7814,PNT,3.0,RCS,0,--,Noisy data,0.0000,GALAXY,1.0,3,0.7814,0.7814,--| No signal| Noisy data,3,none


In [73]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.7814
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. poor snr. flag 1'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
162,35185741956913486,-1,0.33.0.dev3890,0,GALAXY,0.7814,CBA,1.0,--,--,...,1.952842,0.7814,GALAXY,1.0,3,0.7814,0.7814,--| No signal| Noisy data,3,disagreement in flag. poor snr. flag 1
212,35185741956913486,-1,0.33.0.dev3890,0,GALAXY,0.7814,ACE,1.0,--,--,...,1.952842,0.7814,GALAXY,1.0,3,0.7814,0.7814,--| No signal| Noisy data,3,disagreement in flag. poor snr. flag 1
262,35185741956913486,-1,0.33.0.dev3890,0,GALAXY,0.7814,PNT,1.0,RCS,0,...,1.952842,0.7814,GALAXY,1.0,3,0.7814,0.7814,--| No signal| Noisy data,3,disagreement in flag. poor snr. flag 1


conflict ID =  18


In [74]:
#copy this text to a new cell to display the conflict
conflict_id=19
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
163,35185741961101345,GALAXY,0.7812,CBA,0.0,--,--,--,--,0.7812,GALAXY,1.0,3,0.7812,0.7812,--| No signal| Noisy data,3,none
213,35185741961101345,GALAXY,0.7812,ACE,0.0,--,--,--,No signal,0.7812,GALAXY,1.0,3,0.7812,0.7812,--| No signal| Noisy data,3,none
263,35185741961101345,GALAXY,0.7812,PNT,3.0,RCS,0,--,Noisy data,0.0000,GALAXY,1.0,3,0.7812,0.7812,--| No signal| Noisy data,3,none


In [75]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.7812
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. poor snr. flag 1'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
163,35185741961101345,-1,0.33.0.dev3890,0,GALAXY,0.7812,CBA,1.0,--,--,...,8.200933,0.7812,GALAXY,1.0,3,0.7812,0.7812,--| No signal| Noisy data,3,disagreement in flag. poor snr. flag 1
213,35185741961101345,-1,0.33.0.dev3890,0,GALAXY,0.7812,ACE,1.0,--,--,...,8.200933,0.7812,GALAXY,1.0,3,0.7812,0.7812,--| No signal| Noisy data,3,disagreement in flag. poor snr. flag 1
263,35185741961101345,-1,0.33.0.dev3890,0,GALAXY,0.7812,PNT,1.0,RCS,0,...,8.200933,0.7812,GALAXY,1.0,3,0.7812,0.7812,--| No signal| Noisy data,3,disagreement in flag. poor snr. flag 1


conflict ID =  19


In [78]:
#copy this text to a new cell to display the conflict
conflict_id=20
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
164,35185741961101395,QSO,0.862,CBA,1.0,--,--,--,--,0.862,QSO,2.0,2,0.862,0.862,--|--| Noisy data,3,none
214,35185741961101395,QSO,0.862,ACE,2.0,--,--,--,--,0.862,QSO,2.0,2,0.862,0.862,--|--| Noisy data,3,none
264,35185741961101395,QSO,0.862,PNT,3.0,RCS,0,--,Noisy data,0.000,QSO,2.0,2,0.862,0.862,--|--| Noisy data,3,none


In [79]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.862
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. poor signal, flag 1. not a QSO.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
164,35185741961101395,-1,0.33.0.dev3890,0,QSO,0.862,CBA,1.0,--,--,...,0.535343,0.862,GALAXY,2.0,2,0.862,0.862,--|--| Noisy data,3,"disagreement in flag. poor signal, flag 1. not..."
214,35185741961101395,-1,0.33.0.dev3890,0,QSO,0.862,ACE,1.0,--,--,...,0.535343,0.862,GALAXY,2.0,2,0.862,0.862,--|--| Noisy data,3,"disagreement in flag. poor signal, flag 1. not..."
264,35185741961101395,-1,0.33.0.dev3890,0,QSO,0.862,PNT,1.0,RCS,0,...,0.535343,0.862,GALAXY,2.0,2,0.862,0.862,--|--| Noisy data,3,"disagreement in flag. poor signal, flag 1. not..."


conflict ID =  20


In [80]:
#copy this text to a new cell to display the conflict
conflict_id=21
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
167,35185741961101910,GALAXY,1.2386,CBA,4.0,--,--,--,[OII] Balmer abs features,1.2386,GALAXY,3.333333,2,1.2386,1.2386,[OII] Balmer abs features|--| Noisy data,3,none
217,35185741961101910,GALAXY,1.2386,ACE,4.0,--,--,--,--,1.2386,GALAXY,3.333333,2,1.2386,1.2386,[OII] Balmer abs features|--| Noisy data,3,none
267,35185741961101910,GALAXY,1.2386,PNT,2.0,RCS,0,--,Noisy data,0.0000,GALAXY,3.333333,2,1.2386,1.2386,[OII] Balmer abs features|--| Noisy data,3,none


In [81]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
167,35185741961101910,-1,0.33.0.dev3890,0,GALAXY,1.2386,CBA,3.333333,--,--,...,195.138836,1.2386,GALAXY,3.333333,2,1.2386,1.2386,[OII] Balmer abs features|--| Noisy data,3,"flag disagreement, take mean."
217,35185741961101910,-1,0.33.0.dev3890,0,GALAXY,1.2386,ACE,3.333333,--,--,...,195.138836,1.2386,GALAXY,3.333333,2,1.2386,1.2386,[OII] Balmer abs features|--| Noisy data,3,"flag disagreement, take mean."
267,35185741961101910,-1,0.33.0.dev3890,0,GALAXY,1.2386,PNT,3.333333,RCS,0,...,195.138836,0.0000,GALAXY,3.333333,2,1.2386,1.2386,[OII] Balmer abs features|--| Noisy data,3,"flag disagreement, take mean."


conflict ID =  21


In [84]:
#copy this text to a new cell to display the conflict
conflict_id=22
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
168,35185741961101954,GALAXY,0.9568,CBA,4.0,--,--,--,[OII] Balmer abs lines,0.9568,GALAXY,3.333333,2,0.9568,0.0,[OII] Balmer abs lines|--| Noisy data only OI...,3,none
218,35185741961101954,GALAXY,0.9568,ACE,4.0,--,--,--,--,0.9568,GALAXY,3.333333,2,0.9568,0.0,[OII] Balmer abs lines|--| Noisy data only OI...,3,none
268,35185741961101954,GALAXY,0.9568,PNT,2.0,--,0.9568,GALAXY,Noisy data only OII doublet is resolved,0.9568,GALAXY,3.333333,2,0.9568,0.0,[OII] Balmer abs lines|--| Noisy data only OI...,3,none


In [85]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
168,35185741961101954,-1,0.33.0.dev3890,0,GALAXY,0.9568,CBA,3.333333,--,--,...,93.327755,0.9568,GALAXY,3.333333,2,0.0,0.9568,[OII] Balmer abs lines|--| Noisy data only OI...,3,"flag disagreement, take mean."
218,35185741961101954,-1,0.33.0.dev3890,0,GALAXY,0.9568,ACE,3.333333,--,--,...,93.327755,0.9568,GALAXY,3.333333,2,0.0,0.9568,[OII] Balmer abs lines|--| Noisy data only OI...,3,"flag disagreement, take mean."
268,35185741961101954,-1,0.33.0.dev3890,0,GALAXY,0.9568,PNT,3.333333,--,0.9568,...,93.327755,0.9568,GALAXY,3.333333,2,0.0,0.9568,[OII] Balmer abs lines|--| Noisy data only OI...,3,"flag disagreement, take mean."


conflict ID =  22


In [88]:
#copy this text to a new cell to display the conflict
conflict_id=23
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
172,35185741961103043,GALAXY,1.0901,CBA,2.0,--,--,--,Possible [OII],1.0901,GALAXY,2.333333,3,1.0901,0.0,Possible [OII]|--| Noisy data only OII line i...,3,none
222,35185741961103043,GALAXY,1.0901,ACE,4.0,--,--,--,--,1.0901,GALAXY,2.333333,3,1.0901,0.0,Possible [OII]|--| Noisy data only OII line i...,3,none
272,35185741961103043,GALAXY,1.0901,PNT,1.0,--,1.0901,GALAXY,Noisy data only OII line is resolved at λobs~...,1.0901,GALAXY,2.333333,3,1.0901,0.0,Possible [OII]|--| Noisy data only OII line i...,3,none


In [89]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. broadened but unresolved line. poor sky subtraction.z not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
172,35185741961103043,-1,0.33.0.dev3890,0,GALAXY,1.0901,CBA,2.333333,--,--,...,58.574658,1.0901,GALAXY,2.333333,3,0.0,1.0901,Possible [OII]|--| Noisy data only OII line i...,3,flag disagreement. broadened but unresolved li...
222,35185741961103043,-1,0.33.0.dev3890,0,GALAXY,1.0901,ACE,2.333333,--,--,...,58.574658,1.0901,GALAXY,2.333333,3,0.0,1.0901,Possible [OII]|--| Noisy data only OII line i...,3,flag disagreement. broadened but unresolved li...
272,35185741961103043,-1,0.33.0.dev3890,0,GALAXY,1.0901,PNT,2.333333,--,1.0901,...,58.574658,1.0901,GALAXY,2.333333,3,0.0,1.0901,Possible [OII]|--| Noisy data only OII line i...,3,flag disagreement. broadened but unresolved li...


conflict ID =  23


In [90]:
#copy this text to a new cell to display the conflict
conflict_id=24
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
180,35185741961104153,GALAXY,0.6413,CBA,1.0,--,--,--,--,0.6413,GALAXY,2.333333,3,0.6413,0.037105,--| Weak OII and noisy continuum| Noisy data,3,none
230,35185741961104153,GALAXY,0.6413,ACE,2.0,R,0.7022,GALAXY,Weak OII and noisy continuum,0.7022,GALAXY,2.333333,3,0.6413,0.037105,--| Weak OII and noisy continuum| Noisy data,3,none
280,35185741961104153,GALAXY,0.6413,PNT,4.0,RCS,--,--,Noisy data,0.6413,GALAXY,2.333333,3,0.6413,0.037105,--| Weak OII and noisy continuum| Noisy data,3,none


In [91]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
180,35185741961104153,-1,0.33.0.dev3890,0,GALAXY,0.6413,CBA,2.333333,--,--,...,4.906413,0.6413,GALAXY,2.333333,3,0.037105,0.6413,--| Weak OII and noisy continuum| Noisy data,3,"flag disagreement, take mean."
230,35185741961104153,-1,0.33.0.dev3890,0,GALAXY,0.6413,ACE,2.333333,R,0.7022,...,4.906413,0.7022,GALAXY,2.333333,3,0.037105,0.6413,--| Weak OII and noisy continuum| Noisy data,3,"flag disagreement, take mean."
280,35185741961104153,-1,0.33.0.dev3890,0,GALAXY,0.6413,PNT,2.333333,RCS,--,...,4.906413,0.6413,GALAXY,2.333333,3,0.037105,0.6413,--| Weak OII and noisy continuum| Noisy data,3,"flag disagreement, take mean."


conflict ID =  24


In [92]:
#copy this text to a new cell to display the conflict
conflict_id=25
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
181,35185741961104546,GALAXY,1.3554,CBA,1.0,--,--,--,--,1.3554,GALAXY,1.666667,4,1.3554,1.3554,--| No signal|Noisy data,3,none
231,35185741961104546,GALAXY,1.3554,ACE,0.0,--,--,--,No signal,1.3554,GALAXY,1.666667,4,1.3554,1.3554,--| No signal|Noisy data,3,none
281,35185741961104546,GALAXY,1.3554,PNT,4.0,RCS,0,--,Noisy data,0.0000,GALAXY,1.666667,4,1.3554,1.3554,--| No signal|Noisy data,3,none


In [93]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.3554
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. no secure features. flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
181,35185741961104546,-1,0.33.0.dev3890,0,GALAXY,1.3554,CBA,1.0,--,--,...,1.907525,1.3554,GALAXY,1.666667,4,1.3554,1.3554,--| No signal|Noisy data,3,flag disagreement. no secure features. flag 1.
231,35185741961104546,-1,0.33.0.dev3890,0,GALAXY,1.3554,ACE,1.0,--,--,...,1.907525,1.3554,GALAXY,1.666667,4,1.3554,1.3554,--| No signal|Noisy data,3,flag disagreement. no secure features. flag 1.
281,35185741961104546,-1,0.33.0.dev3890,0,GALAXY,1.3554,PNT,1.0,RCS,0,...,1.907525,1.3554,GALAXY,1.666667,4,1.3554,1.3554,--| No signal|Noisy data,3,flag disagreement. no secure features. flag 1.


conflict ID =  25


In [96]:
#copy this text to a new cell to display the conflict
conflict_id=26
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
302,35185741961105562,GALAXY,0.7655,ASC,0.0,S,--,--,--,0.7655,GALAXY,1.0,3,0.7655,1.010705,--| dubious fit (all features in noise) + nosi...,3,none
352,35185741961105562,GALAXY,0.7655,VRK,0.0,RS,--,--,dubious fit (all features in noise) + nosiy s...,0.7655,GALAXY,1.0,3,0.7655,1.010705,--| dubious fit (all features in noise) + nosi...,3,none
402,35185741961105562,GALAXY,0.7655,ACE,3.0,R,2.5499,QSO,Very strong Lyman alpha,2.5499,QSO,1.0,3,0.7655,1.010705,--| dubious fit (all features in noise) + nosi...,3,none


In [97]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 2.5499
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'QSO'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'no continuum. 1/3 inspectors identifies strong line as LyA from QSO. take answer with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
302,35185741961105562,-1,0.33.0.dev3890,0,GALAXY,0.7655,ASC,2.0,S,--,...,0.513559,2.5499,QSO,1.0,3,1.010705,0.7655,--| dubious fit (all features in noise) + nosi...,3,no continuum. 1/3 inspectors identifies strong...
352,35185741961105562,-1,0.33.0.dev3890,0,GALAXY,0.7655,VRK,2.0,RS,--,...,0.513559,2.5499,QSO,1.0,3,1.010705,0.7655,--| dubious fit (all features in noise) + nosi...,3,no continuum. 1/3 inspectors identifies strong...
402,35185741961105562,-1,0.33.0.dev3890,0,GALAXY,0.7655,ACE,2.0,R,2.5499,...,0.513559,2.5499,QSO,1.0,3,1.010705,0.7655,--| dubious fit (all features in noise) + nosi...,3,no continuum. 1/3 inspectors identifies strong...


conflict ID =  26


In [98]:
#copy this text to a new cell to display the conflict
conflict_id=27
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
310,35185741965295921,QSO,2.8379,ASC,3.0,--,--,--,Probable Lya and possible C III] but C IV cut...,2.8379,QSO,2.0,5,2.8379,0.004351,Probable Lya and possible C III] but C IV cut...,3,none
360,35185741965295921,QSO,2.8379,VRK,4.0,--,--,--,clear QSO spectrum,2.8379,QSO,2.0,5,2.8379,0.004351,Probable Lya and possible C III] but C IV cut...,3,none
410,35185741965295921,QSO,2.8379,ACE,-1.0,R,2.8546,QSO,QSO template slightly misfits redshift,2.8546,QSO,2.0,5,2.8379,0.004351,Probable Lya and possible C III] but C IV cut...,3,none


In [99]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'one inspector forgot to set flag. secure z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
310,35185741965295921,-1,0.33.0.dev3890,0,QSO,2.8379,ASC,4.0,--,--,...,137.350605,2.8379,QSO,2.0,5,0.004351,2.8379,Probable Lya and possible C III] but C IV cut...,3,one inspector forgot to set flag. secure z.
360,35185741965295921,-1,0.33.0.dev3890,0,QSO,2.8379,VRK,4.0,--,--,...,137.350605,2.8379,QSO,2.0,5,0.004351,2.8379,Probable Lya and possible C III] but C IV cut...,3,one inspector forgot to set flag. secure z.
410,35185741965295921,-1,0.33.0.dev3890,0,QSO,2.8379,ACE,4.0,R,2.8546,...,137.350605,2.8546,QSO,2.0,5,0.004351,2.8379,Probable Lya and possible C III] but C IV cut...,3,one inspector forgot to set flag. secure z.


conflict ID =  27


In [102]:
#copy this text to a new cell to display the conflict
conflict_id= 28
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
321,35185741965297527,GALAXY,1.5495,ASC,3.0,S,--,--,[O II] unresolved due to noise,1.5495,GALAXY,2.0,3,1.5495,0.0,[O II] unresolved due to noise|dubious fit (b...,3,none
371,35185741965297527,GALAXY,1.5495,VRK,0.0,RS,--,--,dubious fit (broad doublet in skyline region) ...,1.5495,GALAXY,2.0,3,1.5495,0.0,[O II] unresolved due to noise|dubious fit (b...,3,none
421,35185741965297527,GALAXY,1.5495,ACE,3.0,--,--,--,--,1.5495,GALAXY,2.0,3,1.5495,0.0,[O II] unresolved due to noise|dubious fit (b...,3,none


In [103]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.5495
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. broadened but unresolved line. poor sky subtraction. z not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
321,35185741965297527,-1,0.33.0.dev3890,0,GALAXY,1.5495,ASC,2.0,S,--,...,142.132984,1.5495,GALAXY,2.0,3,0.0,1.5495,[O II] unresolved due to noise|dubious fit (b...,3,disagreement in flag. broadened but unresolved...
371,35185741965297527,-1,0.33.0.dev3890,0,GALAXY,1.5495,VRK,2.0,RS,--,...,142.132984,1.5495,GALAXY,2.0,3,0.0,1.5495,[O II] unresolved due to noise|dubious fit (b...,3,disagreement in flag. broadened but unresolved...
421,35185741965297527,-1,0.33.0.dev3890,0,GALAXY,1.5495,ACE,2.0,--,--,...,142.132984,1.5495,GALAXY,2.0,3,0.0,1.5495,[O II] unresolved due to noise|dubious fit (b...,3,disagreement in flag. broadened but unresolved...


conflict ID =  28


In [106]:
#copy this text to a new cell to display the conflict
conflict_id=29
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
324,35185741965298041,GALAXY,0.1029,ASC,0.0,S,--,--,--,0.1029,GALAXY,1.666667,3,0.1029,2.066733,--|dubious fit (one strong broad feature at 41...,3,none
374,35185741965298041,GALAXY,0.1029,VRK,2.0,R,--,--,dubious fit (one strong broad feature at 4100A...,0.1029,GALAXY,1.666667,3,0.1029,2.066733,--|dubious fit (one strong broad feature at 41...,3,none
424,35185741965298041,GALAXY,0.1029,ACE,3.0,--,2.3823,QSO,Based on strong Lyman alpha,2.3823,QSO,1.666667,3,0.1029,2.066733,--|dubious fit (one strong broad feature at 41...,3,none


In [107]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 2.3823
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'QSO'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'no strong features but a single emission line. one inspector identifies as LyA. take VI z with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
324,35185741965298041,-1,0.33.0.dev3890,0,GALAXY,0.1029,ASC,2.0,S,--,...,1.354331,2.3823,QSO,1.666667,3,2.066733,0.1029,--|dubious fit (one strong broad feature at 41...,3,no strong features but a single emission line....
374,35185741965298041,-1,0.33.0.dev3890,0,GALAXY,0.1029,VRK,2.0,R,--,...,1.354331,2.3823,QSO,1.666667,3,2.066733,0.1029,--|dubious fit (one strong broad feature at 41...,3,no strong features but a single emission line....
424,35185741965298041,-1,0.33.0.dev3890,0,GALAXY,0.1029,ACE,2.0,--,2.3823,...,1.354331,2.3823,QSO,1.666667,3,2.066733,0.1029,--|dubious fit (one strong broad feature at 41...,3,no strong features but a single emission line....


conflict ID =  29


In [111]:
#copy this text to a new cell to display the conflict
conflict_id=30
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
344,35185741969490082,GALAXY,1.3515,ASC,0.0,--,--,--,--,1.3515,GALAXY,1.0,3,1.3515,0.0,--|--| OII on sky line not plausible,3,none
394,35185741969490082,GALAXY,1.3515,VRK,3.0,--,--,--,--,1.3515,GALAXY,1.0,3,1.3515,0.0,--|--| OII on sky line not plausible,3,none
444,35185741969490082,GALAXY,1.3515,ACE,0.0,--,--,--,OII on sky line not plausible,1.3515,GALAXY,1.0,3,1.3515,0.0,--|--| OII on sky line not plausible,3,none


In [112]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'poor sky subtraction. doublet on skylines. not secure. flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
344,35185741969490082,-1,0.33.0.dev3890,0,GALAXY,1.3515,ASC,1.0,--,--,...,26.390474,1.3515,GALAXY,1.0,3,0.0,1.3515,--|--| OII on sky line not plausible,3,poor sky subtraction. doublet on skylines. not...
394,35185741969490082,-1,0.33.0.dev3890,0,GALAXY,1.3515,VRK,1.0,--,--,...,26.390474,1.3515,GALAXY,1.0,3,0.0,1.3515,--|--| OII on sky line not plausible,3,poor sky subtraction. doublet on skylines. not...
444,35185741969490082,-1,0.33.0.dev3890,0,GALAXY,1.3515,ACE,1.0,--,--,...,26.390474,1.3515,GALAXY,1.0,3,0.0,1.3515,--|--| OII on sky line not plausible,3,poor sky subtraction. doublet on skylines. not...


conflict ID =  30


In [115]:
#copy this text to a new cell to display the conflict
conflict_id= 31
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
345,35185741969490342,QSO,1.9424,ASC,1.0,--,--,--,Possible C IV,1.9424,QSO,2.0,2,1.9424,0.0,Possible C IV|not an ELG spectrum + could be ...,3,none
395,35185741969490342,QSO,1.9424,VRK,3.0,--,--,--,not an ELG spectrum + could be a QSO,1.9424,QSO,2.0,2,1.9424,0.0,Possible C IV|not an ELG spectrum + could be ...,3,none
445,35185741969490342,QSO,1.9424,ACE,2.0,--,--,--,Broad line in the blue low signal to noise,1.9424,QSO,2.0,2,1.9424,0.0,Possible C IV|not an ELG spectrum + could be ...,3,none


In [116]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.Not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
345,35185741969490342,-1,0.33.0.dev3890,0,QSO,1.9424,ASC,2.0,--,--,...,4.072479,1.9424,QSO,2.0,2,0.0,1.9424,Possible C IV|not an ELG spectrum + could be ...,3,"flag disagreement, take mean.Not secure."
395,35185741969490342,-1,0.33.0.dev3890,0,QSO,1.9424,VRK,2.0,--,--,...,4.072479,1.9424,QSO,2.0,2,0.0,1.9424,Possible C IV|not an ELG spectrum + could be ...,3,"flag disagreement, take mean.Not secure."
445,35185741969490342,-1,0.33.0.dev3890,0,QSO,1.9424,ACE,2.0,--,--,...,4.072479,1.9424,QSO,2.0,2,0.0,1.9424,Possible C IV|not an ELG spectrum + could be ...,3,"flag disagreement, take mean.Not secure."


conflict ID =  31


### and so on...

We should now recompute the conflicts, and not find any.

In [118]:
vi_conflict = VI.find_conflicts(vi_gp)

In [119]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185741952713123 35185741961101910 35185741961104153 35185741965295921]
Total number of conflicts to resolve:  4


Display anything that was missed (if "Total number of conflicts" isn't zero) and resolve!

In [120]:
#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
29,35185741952713123,GALAXY,0.2835,PNT,2.5,RCS,0.2835,STAR,Noisy spectra,1.321,STAR,2.333333,1,0.2835,0.808337,Noisy spectra| I suggest this is bad z fit by...,3,disagreement in flag and z. 2/3 inspectors sug...
78,35185741952713123,GALAXY,0.2835,CBA,2.5,R,1.3209,--,I suggest this is bad z fit by RR [OII] doub...,1.321,GALAXY,2.333333,1,0.2835,0.808337,Noisy spectra| I suggest this is bad z fit by...,3,disagreement in flag and z. 2/3 inspectors sug...
128,35185741952713123,GALAXY,0.2835,ace,2.5,--,1.3210,GALAXY,Redrock misses and obvious OII line,1.321,GALAXY,2.333333,1,0.2835,0.808337,Noisy spectra| I suggest this is bad z fit by...,3,disagreement in flag and z. 2/3 inspectors sug...


In [121]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
29,35185741952713123,-1,0.33.0.dev3890,0,GALAXY,0.2835,PNT,2.5,RCS,0.2835,...,4.530503,1.321,GALAXY,2.333333,1,0.808337,0.2835,Noisy spectra| I suggest this is bad z fit by...,3,disagreement in flag and z. 2/3 inspectors sug...
78,35185741952713123,-1,0.33.0.dev3890,0,GALAXY,0.2835,CBA,2.5,R,1.3209,...,4.530503,1.321,GALAXY,2.333333,1,0.808337,0.2835,Noisy spectra| I suggest this is bad z fit by...,3,disagreement in flag and z. 2/3 inspectors sug...
128,35185741952713123,-1,0.33.0.dev3890,0,GALAXY,0.2835,ace,2.5,--,1.3210,...,4.530503,1.321,GALAXY,2.333333,1,0.808337,0.2835,Noisy spectra| I suggest this is bad z fit by...,3,disagreement in flag and z. 2/3 inspectors sug...


conflict ID =  0


In [122]:
#copy this text to a new cell to display the conflict
conflict_id=1
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
167,35185741961101910,GALAXY,1.2386,CBA,3.333333,--,--,--,[OII] Balmer abs features,1.2386,GALAXY,3.333333,2,1.2386,1.2386,[OII] Balmer abs features|--| Noisy data,3,"flag disagreement, take mean."
217,35185741961101910,GALAXY,1.2386,ACE,3.333333,--,--,--,--,1.2386,GALAXY,3.333333,2,1.2386,1.2386,[OII] Balmer abs features|--| Noisy data,3,"flag disagreement, take mean."
267,35185741961101910,GALAXY,1.2386,PNT,3.333333,RCS,0,--,Noisy data,0.0000,GALAXY,3.333333,2,1.2386,1.2386,[OII] Balmer abs features|--| Noisy data,3,"flag disagreement, take mean."


In [123]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.2386
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
167,35185741961101910,-1,0.33.0.dev3890,0,GALAXY,1.2386,CBA,3.333333,--,--,...,195.138836,1.2386,GALAXY,3.333333,2,1.2386,1.2386,[OII] Balmer abs features|--| Noisy data,3,"flag disagreement, take mean."
217,35185741961101910,-1,0.33.0.dev3890,0,GALAXY,1.2386,ACE,3.333333,--,--,...,195.138836,1.2386,GALAXY,3.333333,2,1.2386,1.2386,[OII] Balmer abs features|--| Noisy data,3,"flag disagreement, take mean."
267,35185741961101910,-1,0.33.0.dev3890,0,GALAXY,1.2386,PNT,3.333333,RCS,0,...,195.138836,1.2386,GALAXY,3.333333,2,1.2386,1.2386,[OII] Balmer abs features|--| Noisy data,3,"flag disagreement, take mean."


conflict ID =  1


In [124]:
#copy this text to a new cell to display the conflict
conflict_id=2
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
180,35185741961104153,GALAXY,0.6413,CBA,2.333333,--,--,--,--,0.6413,GALAXY,2.333333,3,0.6413,0.037105,--| Weak OII and noisy continuum| Noisy data,3,"flag disagreement, take mean."
230,35185741961104153,GALAXY,0.6413,ACE,2.333333,R,0.7022,GALAXY,Weak OII and noisy continuum,0.7022,GALAXY,2.333333,3,0.6413,0.037105,--| Weak OII and noisy continuum| Noisy data,3,"flag disagreement, take mean."
280,35185741961104153,GALAXY,0.6413,PNT,2.333333,RCS,--,--,Noisy data,0.6413,GALAXY,2.333333,3,0.6413,0.037105,--| Weak OII and noisy continuum| Noisy data,3,"flag disagreement, take mean."


In [125]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.6413
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
180,35185741961104153,-1,0.33.0.dev3890,0,GALAXY,0.6413,CBA,2.333333,--,--,...,4.906413,0.6413,GALAXY,2.333333,3,0.037105,0.6413,--| Weak OII and noisy continuum| Noisy data,3,"flag disagreement, take mean."
230,35185741961104153,-1,0.33.0.dev3890,0,GALAXY,0.6413,ACE,2.333333,R,0.7022,...,4.906413,0.6413,GALAXY,2.333333,3,0.037105,0.6413,--| Weak OII and noisy continuum| Noisy data,3,"flag disagreement, take mean."
280,35185741961104153,-1,0.33.0.dev3890,0,GALAXY,0.6413,PNT,2.333333,RCS,--,...,4.906413,0.6413,GALAXY,2.333333,3,0.037105,0.6413,--| Weak OII and noisy continuum| Noisy data,3,"flag disagreement, take mean."


conflict ID =  2


In [128]:
#copy this text to a new cell to display the conflict
conflict_id=3
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
310,35185741965295921,QSO,2.8379,ASC,4.0,--,--,--,Probable Lya and possible C III] but C IV cut...,2.8379,QSO,2.0,5,2.8379,0.004351,Probable Lya and possible C III] but C IV cut...,3,one inspector forgot to set flag. secure z.
360,35185741965295921,QSO,2.8379,VRK,4.0,--,--,--,clear QSO spectrum,2.8379,QSO,2.0,5,2.8379,0.004351,Probable Lya and possible C III] but C IV cut...,3,one inspector forgot to set flag. secure z.
410,35185741965295921,QSO,2.8379,ACE,4.0,R,2.8546,QSO,QSO template slightly misfits redshift,2.8546,QSO,2.0,5,2.8379,0.004351,Probable Lya and possible C III] but C IV cut...,3,one inspector forgot to set flag. secure z.


In [129]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = (2.8546+2.8379)/2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
310,35185741965295921,-1,0.33.0.dev3890,0,QSO,2.8379,ASC,4.0,--,--,...,137.350605,2.84625,QSO,2.0,5,0.004351,2.8379,Probable Lya and possible C III] but C IV cut...,3,one inspector forgot to set flag. secure z.
360,35185741965295921,-1,0.33.0.dev3890,0,QSO,2.8379,VRK,4.0,--,--,...,137.350605,2.84625,QSO,2.0,5,0.004351,2.8379,Probable Lya and possible C III] but C IV cut...,3,one inspector forgot to set flag. secure z.
410,35185741965295921,-1,0.33.0.dev3890,0,QSO,2.8379,ACE,4.0,R,2.8546,...,137.350605,2.84625,QSO,2.0,5,0.004351,2.8379,Probable Lya and possible C III] but C IV cut...,3,one inspector forgot to set flag. secure z.


conflict ID =  3


Let's check the number of conflicts again.

In [131]:
vi_conflict = VI.find_conflicts(vi_gp)

In [132]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


## Woohoo!

## Finally
we need to recompute vi_combined_flag and vi_combined_z, after everything is resolved.

In [133]:
vi['vi_combined_flag'] = vi.groupby('TargetID')['VI class'].transform('mean')
vi['vi_combined_z'] = vi.groupby('TargetID')['best redshift'].transform('mean')


## Now we prepare to write to file. 

### The important columns for the truth table construction are **vi_combined_flag** and **vi_combined_z** 

The truth table should **take the redhift value in vi_combined_z** for all objects with **vi_combined_flag >= 2.5** (for discussion, but this catches the cases where there is some disagreement on the 2/3 boundary).

After the merging process, those two columns will be the same for ALL of the members of each group by TargetID, so it doesn't matter which member we write to file. Taking the 1st one just makes this easy. 


In [134]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff', 'dz',
       'vi_combined_z', 'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')

In [136]:
print(VI_dir+save_filename)

/global/homes/r/rtojeiro/SV/VI_files/SV0/ELG/truth_table_ELGs_sets2_4_new.txt


In [135]:
vi_gp['Redrock z', 'Redrock spectype', 'best redshift' , 'best spectype', 'vi_combined_flag',  
      'all VI comments', 'merger comment', 'N_VI', 'DELTACHI2','FIBER'].first().to_csv(VI_dir+save_filename)

Check that merged file reads in OK - check comments

In [137]:
merged_file = pd.read_csv(VI_dir+save_filename)

In [138]:
merged_file.keys()

Index(['TargetID', 'Redrock z', 'Redrock spectype', 'best redshift',
       'best spectype', 'vi_combined_flag', 'all VI comments',
       'merger comment', 'N_VI', 'DELTACHI2', 'FIBER'],
      dtype='object')

In [139]:
merged_file

,TargetID,Redrock z,Redrock spectype,best redshift,best spectype,vi_combined_flag,all VI comments,merger comment,N_VI,DELTACHI2,FIBER
0,35185735929695223,1.0936,GALAXY,1.0936,GALAXY,2.333333,Difficult to asses. I think it is noisy spect...,disagreement in flag. broadened but unresolved...,3,46.116570,328
1,35185735929695459,0.8155,GALAXY,0.8155,GALAXY,4.000000,--| [OII] Hb [OIII]|--,none,3,959.128342,333
2,35185735929695590,1.5142,GALAXY,0.0000,GALAXY,0.333333,Noisy spectra| Bad sky subtraction not [OII]|...,none,3,1.790732,343
3,35185735929695765,1.2587,GALAXY,1.2587,GALAXY,4.000000,--| [OII] doublet|--,none,3,627.763939,303
4,35185735929695797,1.1690,GALAXY,1.1690,GALAXY,4.000000,--| [OII] doublet|--,none,3,554.882680,340
...,...,...,...,...,...,...,...,...,...,...,...
145,35185741965300315,0.8725,GALAXY,0.8725,GALAXY,4.000000,--|--|--,none,3,227.709846,493
146,35185741969490082,1.3515,GALAXY,1.3515,GALAXY,1.000000,--|--| OII on sky line not plausible,poor sky subtraction. doublet on skylines. not...,3,26.390474,342
147,35185741969490342,1.9424,QSO,1.9424,QSO,2.000000,Possible C IV|not an ELG spectrum + could be ...,"flag disagreement, take mean.Not secure.",3,4.072479,302
148,35185741969490438,1.6280,GALAXY,1.6280,GALAXY,0.333333,--|dubious doublet at the very end of the NIR ...,none,3,0.852903,313


In [140]:
merged_file.loc[9]['merger comment']

'flag disagreement, take mean.'

In [141]:
merged_file.loc[9]['all VI comments']

' Seems like a STAR but it was classified as galaxy| Star spectrum| White dwarf - no a galaxy!'

In [ ]:
VI_dir